In [8]:
pip install git

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement git (from versions: none)
ERROR: No matching distribution found for git


In [10]:
!sudo dnf install git-all

'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
!git clone https://github.com/Horizon2333/imagenet-autoaencoder

'git' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!python tools/generate_list.py --name weather_test --path ./dataset

python: can't open file 'c:\Users\seung\Desktop\auto_image\tools\generate_list.py': [Errno 2] No such file or directory
